In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import bids
from bids import BIDSLayout, BIDSValidator
from tqdm import tqdm

import nilearn as nil
from nilearn.input_data import NiftiMasker
from nilearn.image import resample_to_img, resample_img
from nilearn.datasets import load_mni152_template, load_mni152_brain_mask
from nilearn.glm.first_level.hemodynamic_models import compute_regressor
import nibabel as nib

from skimage.measure import block_reduce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale

from scipy.stats import ttest_1samp, ttest_ind, t, sem
import hbayesdm.models

from concurrent.futures import ProcessPoolExecutor

In [2]:
bids.config.set_option('extension_initial_dot', True)

args = {
    'root': '/data2/project_model_based_fmri/ds000005/',
    'mask_path': '/data2/project_model_based_fmri/ds000005/masking',
    'save_path': '/data2/project_model_based_fmri/ds000005/data',
    
    'dm_model': 'ra_prospect',
    'hrf_model': 'glover',
    'ncore': 32
}

In [3]:
def array2pindex(array, p_value=0.05, flatten=False):
    confidence = 1 - p_value
    flattened_array = array.flatten()
    
    n = len(flattened_array)
    m = np.mean(flattened_array)
    std_err = sem(flattened_array)
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    end = m + h
    
    ret = (flattened_array >= end) if flatten is True else (array >= end)
    return ret

In [4]:
def custom_masking(mask_path, p_value, zoom, smoothing_fwhm, interpolation_func, flatten=False):
    if mask_path is None:
        assert (mask_path is None)
    elif type(mask_path) is str:
        mask_files = [mask_path]
    else:
        if type(mask_path) is not type(Path()):
            mask_path = Path(mask_path)
        mask_files = [file for file in mask_path.glob('*.nii.gz')]

    mni152_mask = load_mni152_brain_mask()

    m = array2pindex(nib.load(mask_files[0]).get_fdata(), p_value, flatten)

    if len(mask_files) == 1:
        return m
    else:
        for i in range(1, len(mask_files)):
            m |= array2pindex(nib.load(mask_files[i]).get_fdata(), p_value, flatten)

    m = block_reduce(m, zoom, interpolation_func)
    m = 1 * (m > 0)

    m_true = np.array([i for i, v in enumerate(m.flatten()) if v != 0])
    masked_data = nib.Nifti1Image(m, affine=mni152_mask.affine)
    masker = NiftiMasker(mask_img=masked_data,
                         standardize=True,
                         smoothing_fwhm=smoothing_fwhm)

    return masked_data, masker, m_true

In [5]:
def image_preprocess(params):
    image_filepath, confounds, motion_confounds, masker, masked_data, num = params
    
    if confounds is not None:
        confounds = pd.read_table(confounds, sep='\t')
        confounds = confounds[motion_confounds]
        confounds = confounds.to_numpy()

    fmri_masked = resample_to_img(image_filepath, masked_data)
    fmri_masked = masker.fit_transform(fmri_masked, confounds=confounds)
    
    return fmri_masked

In [14]:
def bids_preprocess(root, mask_path, 
                    save_path=None,
                    single_file=False,
                    zoom=(1, 1, 1),
                    smoothing_fwhm=6,
                    interpolation_func=np.mean,
                    motion_confounds=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z'],
                    p_value=0.05,
                    task_name='task-zero',
                    ncore=os.cpu_count()):

    masked_data, masker, m_true = custom_masking(mask_path, p_value, zoom, smoothing_fwhm, interpolation_func)
    print(masked_data.get_fdata().shape)
    print(m_true.shape)
    print('...preprocessing')
    
    print('...loading and validating BIDS')
    layout = BIDSLayout(root, derivatives=True)
    nii_layout = layout.derivatives['fMRIPrep'].get(return_type='file', suffix='bold', extension='nii.gz')
    reg_layout = layout.derivatives['fMRIPrep'].get(return_type='file', suffix='regressors', extension='tsv')
    print('...done!')
    
    print('...image processing using %s cores' % ncore)
    params = [[z[0], z[1], motion_confounds, masker, masked_data, i]
            for i, z in enumerate(zip(nii_layout, reg_layout))]

    with ProcessPoolExecutor() as executor:
        X = np.array(list(executor.map(image_preprocess, params)))
    print('...done!')

    if save_path is not None:
        sp = Path(args['save_path'])
        if not sp.exists():
            sp.mkdir()
        
        if single_file:
            np.save(sp / 'X.npy', X)
        else:
            for i in range(X.shape[0]):
                np.save(sp / f'X_{i+1}.npy', X[i])
        np.save(sp / 'restore_map.npy', m_true)
    else:
        pass
    
    return X, m_true

In [15]:
import time
s = time.time()
X, m_true = bids_preprocess(Path(args['root']), Path(args['mask_path']), Path(args['save_path']), single_file=True, zoom=(2, 2, 2))
e = time.time()
print('time elapsed:', e - s)

(46, 55, 46)
(6810,)
...preprocessing
...loading and validating BIDS
...done!
...image processing using 88 cores
...done!
time elapsed: 56.7156138420105


In [9]:
X.shape, m_true.shape

((48, 240, 6810), (6810,))

In [3]:
def preprocess_events(params, df_events, df_events_info):
    for i in range(len(df_events)):
        df_events[i]['run'] = df_events_info[i]['run']
        df_events[i]['subjID'] = df_events_info[i]['subject']
        df_events[i]['gamble'] = df_events[i]['respcat'].apply(lambda x: 1 if x == 1 else 0)
        df_events[i]['cert'] = 0 # certain..?

In [4]:
def calculate_modulation(params, df_events, latent_params):
    # postprep
    for i in range(len(df_events)):
        idx = df_events[i].iloc[0]['subjID']
        df_events[i]['rho'] = latent_params.loc[idx]['rho']
        df_events[i]['lambda'] = latent_params.loc[idx]['lambda']
        df_events[i]['modulation'] = \
            (df_events[i]['gain'] ** df_events[i]['rho']) \
            - (df_events[i]['lambda'] * (df_events[i]['loss'] ** df_events[i]['rho']))

In [5]:
funcs = [preprocess_events, calculate_modulation]

In [6]:
def prepare_events(params, funcs):
    layout = BIDSLayout(params['root'], derivatives=True)
    t_r = layout.get_tr()
    events = layout.get(suffix='events', extension='tsv')
    image_sample = nib.load(
        layout.derivatives['fMRIPrep'].get(
            return_type='file',
            suffix='bold',
            extension='nii.gz')[0]
    )
    n_scans = image_sample.shape[-1]
    
    df_events_list = [event.get_df() for event in events]
    df_events_info = [event.get_entities() for event in events]
    
    if len(df_events_list) != len(df_events_info):
        assert()
    
    funcs[0](params, df_events_list, df_events_info)
    
    ncore = os.cpu_count() - 1
    if 'ncore' in params.keys():
        ncore = params['ncore']
        
    dm_model = getattr(hbayesdm.models, params['dm_model'])(
        data=pd.concat(df_events_list), ncore=ncore)

    funcs[1](params, df_events_list, dm_model.all_ind_pars)
    
    frame_times = t_r * (np.arange(n_scans) + t_r/2)
    
    df_events = pd.concat(df_events_list)
    signals = []
    for name0, group0 in tqdm(df_events.groupby(['subjID'])):
        signal_subject = []
        for name1, group1 in df_events.groupby(['run']):
            exp_condition = np.array(group1[['onset', 'duration', 'modulation']]).T

            signal, name = compute_regressor(
                exp_condition=exp_condition,
                hrf_model=params['hrf_model'],
                frame_times=frame_times)
            signal_subject.append(signal)
        
        signal_subject = np.array(signal_subject)
        reshape_target = signal_subject.shape
        
        normalized_signal = minmax_scale(signal_subject.flatten(), axis=0, copy=True)
        normalized_signal = normalized_signal.reshape(-1, n_scans, 1)
        signals.append(normalized_signal)
    signals = np.array(signals)
    
    if save_path is not None:
        sp = Path(args['save_path'])
        if not sp.exists():
            sp.mkdir()
        np.save(sp / 'y.npy', X)
    else:
        pass
    
    return dm_model, df_events, signals

In [ ]:
a, b, y = prepare_events(args, funcs)

INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl



Model  = ra_prospect
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 32
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 16
 # of (max) trials per subject  = 256

Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl


In [ ]:
y.shape

In [42]:
import pickle
# save
with open('data2.pkl', 'wb') as f:
    pickle.dump([X, y.reshape(48, 240, 1)], f, pickle.HIGHEST_PROTOCOL)

In [45]:
y.shape

(16, 3, 240, 1)

In [44]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras import Sequential, layers, losses, optimizers, datasets
from tensorflow.keras.layers import Dense, BatchNormalization, ReLU
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [45]:
class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(X.shape[0])
        self.on_epoch_end()
        
    # for printing the statistics of the function
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        "Denotes the number of batches per epoch"
        
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):  # index : batch no.
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        niis = [X[i] for i  in indexes]
        targets = [y[i] for i in indexes]
        niis = np.array(niis)
        targets = np.array(targets)

        return niis, targets  # return batch

In [46]:
alpha = 0.001
lambda_par = 1
repeat_N = 100
batch_size = 64
epochs = 100

In [47]:
X.shape, c.shape

((11520, 32513), (48, 240, 1))

In [48]:
X = X.reshape(-1, 32513)
y = c.reshape(-1, 1)
print(X1.shape, y1.shape)

(11520, 32513) (11520, 1)


In [1]:
coeffs = []

for repeat_i in range(repeat_N):
    ids = np.arange(X.shape[0])
    train_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=repeat_i)
    train_steps = len(train_ids) // batch_size
    valid_steps = len(test_ids) // batch_size

    X_train = X[train_ids]
    y_train = y[train_ids]
    X_test = X[test_ids]
    y_test = y[test_ids]
    
    train_generator = DataGenerator(X_train, y_train, batch_size, shuffle=True)
    valid_generator = DataGenerator(X_test, y_test, batch_size, shuffle=False)
    
    bst_model_path = f'{repeat_i}_best_elasticnet_alpha{alpha:0.3f}_lambda{lambda_par:0.2f}.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True,monitor='val_loss',mode='min',)
    
    model = Sequential()
    model.add(Dense(1, activation='linear', input_shape=(X.shape[-1],),
                    use_bias=True,
                    kernel_regularizer=l1_l2(lambda_par*alpha,lambda_par*(1-alpha)/2),)) 
    model.compile(loss='mse', optimizer='adam',)
    
    model.fit_generator(generator=train_generator, validation_data=valid_generator,
                        steps_per_epoch=train_steps, validation_steps=valid_steps,
                        e pochs=epochs,
                        verbose=0, callbacks=[EarlyStopping(monitor='val_loss', patience=5), model_checkpoint])
    model.load_weights(bst_model_path)

NameError: name 'repeat_N' is not defined

In [ ]:
    
    pred = model(X_test).numpy()
    mse = ((pred-y_test)**2).mean()
    print(f'INFO [{repeat_i+1}/{repeat_N}] - mse: {mse:.03f}')
    coeff = model.layers[0].get_weights()[0]  
    coeffs.append(coeff)

coeffs = np.array(coeffs)

t_val = ttest_1samp(coeffs,0).statistic.reshape((23, 28, 23))
t_val = resize(t_val,(91,109,91))

t_val *= mask.get_data()
nii_i = nib.Nifti1Image(t_val, affine=mask.affine)

nii_i.to_filename(f'elasticnet_keras_alpha{alpha:0.3f}_lambda{lambda_par:0.2f}.nii')

print(f'INFO [{alpha}]: done! ')